In [1]:
!pip install -q peft transformers datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.0 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import torch
import gc

In [3]:
gc.collect()
torch.cuda.empty_cache()

In [4]:
# 2️⃣ Model ve tokenizer
model_path = "ibm-granite/granite-3.3-2b-instruct"
device = "cuda"

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(model_path)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

In [5]:
# 3️⃣ LoRA konfigürasyonu
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Ana model parametrelerini dondur, sadece LoRA güncellenebilir
for name, param in model.named_parameters():
    if "lora" not in name:
        param.requires_grad = False

model.print_trainable_parameters()  # Sadece LoRA parametreleri görünmeli


trainable params: 2,129,920 || all params: 2,535,669,760 || trainable%: 0.0840


In [6]:
from google.colab import drive
import json

# Google Drive’ı bağla
drive.mount('/content/drive')

# JSON dosyasını oku
with open("/content/drive/MyDrive/data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"Toplam kayıt sayısı: {len(data)}")
print("Örnek kayıt:", data[0])


Mounted at /content/drive
Toplam kayıt sayısı: 51914
Örnek kayıt: {'instruction': "Fransa'nın başkenti nedir?", 'input': '', 'output': "Fransa'nın başkenti Paris'tir."}


In [7]:
# 2️⃣ Dataset objesine dönüştür
dataset = Dataset.from_list(data)
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 51914
})


In [8]:
def preprocess(example):
    # instruction + input birleştir
    prompt = example["instruction"]
    if example.get("input") and example["input"].strip():
        prompt += "\n" + example["input"]

    # Tokenize input
    model_inputs = tokenizer(
        prompt,
        truncation=True,
        max_length=256,
        padding="max_length"
    )

    # Tokenize output (labels)
    labels = tokenizer(
        example["output"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=False)
print("Tokenize edilmiş örnek:", tokenized_dataset[0])



Map:   0%|          | 0/51914 [00:00<?, ? examples/s]

Tokenize edilmiş örnek: {'instruction': "Fransa'nın başkenti nedir?", 'input': '', 'output': "Fransa'nın başkenti Paris'tir.", 'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6082, 693, 83, 25, 96, 8166, 30381, 93, 24918, 310, 318, 476, 49], 'attention_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
training_args = TrainingArguments(
    output_dir="./lora-granite",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100
)


In [ ]:
# 8️⃣ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# ------------------------------
# 9️⃣ Modeli eğit
trainer.train()

/tmp/ipython-input-3615374334.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: karadeniz-furk4n (karadeniz-furk4n-trt) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
test_conv = [{"role": "user", "content": "Türkçede 'yazmak' fiilinin geçmiş zaman çekimlerini yap."}]
input_ids = tokenizer.apply_chat_template(
    test_conv,
    return_tensors="pt",
    return_dict=True,
    add_generation_prompt=True
).to(device)

set_seed(42)
output = model.generate(
    **input_ids,
    max_new_tokens=128
)

prediction = tokenizer.decode(output[0, input_ids["input_ids"].shape[1]:], skip_special_tokens=True)
print(prediction)

Türkçe'de 'yazmak' fiilinin geçmiş zaman çekimleri aşağıdaki gibi olmuştur:

1. **Yazmak (yazmak)**: Bu fiil, Türkçe'de 'yazmak' anlamına gelir. İlk olarak, 'yazmak' fiilinin kullanımı, "yaz" cümlelerinde ve "yazı" anlamına gelir. Bu fiil, "yaz" cümlelerinde "yaz" anlamına
